In [1]:
import sys
import time
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

In [2]:
# Load data
with open('data/non_gene_seqs.pkl', 'rb') as f:
    non_gene = pickle.load(f)
with open('data/gene_seqs.pkl', 'rb') as f:
    gene = pickle.load(f)

# Check class balance
print("# non gene examples: {}".format(len(non_gene)))
print("# gene examples: {}".format(len(gene)))
    
# Check for weird bases
print("Bases in non gene: {}".format(set("".join(non_gene))))
print("Bases in gene: {}".format(set("".join(gene))))

# non gene examples: 19800
# gene examples: 19425
Bases in non gene: {'C', 'T', 'G', 'A'}
Bases in gene: {'C', 'T', 'G', 'A'}


In [3]:
# Form test, train and validation sets (class 0 is non-gene and class 1 is gene)
df_non_gene = pd.DataFrame.from_dict({'sequence':non_gene, 'class': 0})
df_gene = pd.DataFrame.from_dict({'sequence':gene, 'class': 1})
df = pd.concat([df_non_gene, df_gene])
df = df.sample(frac=1)  # randomly shuffle dataframe rows

train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(df, test_size=0.1)
print("Train:{}, test:{}, validation:{}".format(len(train), len(test), len(val)))

Train:35302, test:7845, validation:3923


In [4]:
# Convert to test and train matrices
word_dict = dict(zip(['A', 'C', 'G', 'T'], list(range(4))))

def gene_seq_to_vec(seq, word_dict):
    for key in word_dict:
        seq = seq.replace(key, str(word_dict[key]))
    return list(map(int, list(seq)))

X_train = np.array([gene_seq_to_vec(x, word_dict) for x in train['sequence'].tolist()])
X_test = np.array([gene_seq_to_vec(x, word_dict) for x in test['sequence'].tolist()])
X_val = np.array([gene_seq_to_vec(x, word_dict) for x in val['sequence'].tolist()])

y_train = np.array(train['class'].tolist()).reshape(-1, 1)
y_test = np.array(test['class'].tolist()).reshape(-1, 1)
y_val = np.array(val['class'].tolist()).reshape(-1, 1)

In [10]:
# Initialize TF graph and set training paramaters
tf.reset_default_graph()
cell_units = 128
n_hidden = 64
learning_rate = 0.01
b_size = 100
n_epochs = 1

x = tf.placeholder(tf.float32, [None, 1000, 1])
y = tf.placeholder(tf.float32, [None, 1])
batch_size = tf.placeholder(tf.int32)

def gru_rnn(x, y, cell_units, n_hidden, learning_rate):
    """Build TF computation graph for RNN"""
    W1 = tf.get_variable("W1", [cell_units, n_hidden],
                         initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01))
    W2 = tf.get_variable("W2", [n_hidden, 1],
                         initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01))
    b1 = tf.get_variable("b1", [n_hidden], 
                         initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01))
    b2 = tf.get_variable("b2", [1], 
                         initializer=tf.random_normal_initializer(mean=0.0, stddev=0.01))
    # LSTM layer
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units=cell_units)
    outputs, state = tf.nn.dynamic_rnn(cell=lstm, dtype=tf.float32, inputs=x)

    # Linear layer + ReLU + linear layer
    final_output = outputs[:, -1, :]
    h1 = tf.nn.relu(tf.matmul(final_output, W1) + b1)
    h2 = tf.matmul(h1, W2) + b2
    output = tf.nn.sigmoid(h2)

    # Predict on final output and calculate loss function
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=h2))
    acc = tf.reduce_sum(tf.cast(tf.equal(tf.round(output), y), 'float'))

    # Train RNN
    opt = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    return output, cost, opt, acc

output, cost, opt, acc = gru_rnn(x, y, cell_units, n_hidden, learning_rate)
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
# Split into batches and train
X_train_batch_list = np.array_split(X_train, int(len(train)/b_size))
y_train_batch_list = np.array_split(y_train, int(len(train)/b_size))

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        print("#------- Running for epoch {} of {}".format(epoch+1, n_epochs))
        t0 = time.time()
        for i in range(len(X_train_batch_list)):
            train_dict = {
                x: X_train_batch_list[i].reshape(-1, 1000, 1),
                y: y_train_batch_list[i]
            }
            c, o, a = sess.run([cost, opt, acc], feed_dict=train_dict)
            if i % 10 == 0:
                print('Trained for {} batches - loss = {}, accuracy = {}'.format(i, c, (a/b_size)))
                
    sess.close()

#------- Running for epoch 1 of 1
Trained for 0 batches - loss = 0.6933265328407288, accuracy = 0.46
Trained for 10 batches - loss = 0.6932650804519653, accuracy = 0.46
Trained for 20 batches - loss = 0.6910545229911804, accuracy = 0.57
Trained for 30 batches - loss = 0.6940783858299255, accuracy = 0.5
